In [1]:
import sys
import os
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent))

In [2]:
from loader import Loader
from constant import FeatureType, FileType
from copy import deepcopy
from skimage.measure import compare_nrmse
from keras.models import Model
from keras.optimizers import RMSprop
from keras.layers import Input, LSTM, Dense, RepeatVector
import time
import argparse
import numpy as np
import tensorflow as tf
import hashlib
import json

### Train, Validate, Test

In [3]:
def encoder_decoder(partition, args):
    X_train, y_train = partition['train']
    X_val, y_val = partition['val']

    batch_size = args.batch_size
    epochs = args.epochs
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=args.early_stop)

    with tf.device('/GPU:0'):        
        encoder_inputs = Input(shape=(args.x_frames * 24, len(args.features)))

        encoder = LSTM(args.hid_dim, return_state=True)
        encoder_outputs, state_h, state_c = encoder(encoder_inputs)
        encoder_states = [state_h, state_c]

        repeat_vector = RepeatVector(24)
        repeat_vector_outputs = repeat_vector(encoder_outputs)

        decoder_lstm = LSTM(args.hid_dim, return_sequences=True, return_state=True)
        decoder_outputs, _, _ = decoder_lstm(repeat_vector_outputs, initial_state=encoder_states)

        decoder_dense = Dense(1, activation=args.activation)
        decoder_outputs = decoder_dense(decoder_outputs)

        rmsprop = RMSprop(lr=args.lr)

        model = Model(encoder_inputs, decoder_outputs)
        model.compile(optimizer=rmsprop, loss=tf.keras.losses.MeanSquaredError(), metrics='accuracy')

        history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
              batch_size=batch_size, epochs=epochs, callbacks=[callback])
        
        print(model.summary())

    return model, history

In [4]:
def test(partition, model, args):
    X_test, y_test = partition['test']
    
    y_pred = model.predict(X_test)
    y_pred = y_pred.reshape(y_test.shape[0], y_test.shape[1])

    nrmse = compare_nrmse(y_test, y_pred)
    return y_pred, nrmse

In [5]:
def experiment(args):
    loader = Loader(args)
    partition = loader.get_dataset()
    
    ts = time.time()
    model, history = encoder_decoder(partition, args)
    te = time.time()
    
    print('Took {:2.2f} sec for training the model'.format(te-ts))
    
    y_pred, test_acc = test(partition, model, args)
    
    # ======= Add Result to Dictionary ======= #
    result = {}
    result['train_losses'] = history.history['loss']
    result['val_losses'] = history.history['val_loss']
    result['train_accs'] = history.history['accuracy']
    result['val_accs'] = history.history['val_accuracy']
    result['train_acc'] = np.mean(history.history['accuracy'])
    result['val_acc'] = np.mean(history.history['val_accuracy'])
    result['test_nrmse'] = test_acc
    result['y_test'] = np.asarray(partition['test'][1]).tolist()
    result['y_pred'] = np.asarray(y_pred).tolist()
    
    setattr(args, 'features', str(args.features))
    return vars(args), model, result

### Manage Experiment

In [6]:
def get_filepath(setting, filetype):
    exp_name = setting['exp_name']
    hash_key = hashlib.sha1(str(setting).encode()).hexdigest()[:6]
    
    file_format = ""
    directory = ""
    if filetype == FileType.MODEL:
        file_format = 'h5'
        directory = 'models'
    elif filetype == FileType.RESULT:
        file_format = 'json'
        directory = 'results'
        
    filename = "%s-%s.%s" % (exp_name, hash_key, file_format)
    filepath = os.path.join(Path(os.getcwd()).parent.parent, directory, filename)
    return filepath
        
def save_exp_model(setting, model):
    filepath = get_filepath(setting, FileType.MODEL)
    model.save(filepath)

def save_exp_result(setting, result):
    filepath = get_filepath(setting, FileType.RESULT)
    
    result.update(setting)
    with open(filepath, 'w') as f:
        json.dump(result, f)
        
def load_exp_result(exp_name):
    dir_path = os.path.join(Path(os.getcwd()).parent.parent, "results")
    filenames = [f for f in listdeir(dir_path) if isfile(join(dir_path, f)) if '.json' in f]
    list_result = []
    for filename in filenames:
        if exp_name in filename:
            with open(join(dir_path, filename), 'r') as infile:
                results = json.load(infile)
                list_result.append(results)
    df = pd.DataFrame(list_result)
    return df

In [7]:
# ====== Random Seed Initialization ====== #
seed = 1234
tf.random.set_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.exp_name = "solar power prediction using weather features"

# ====== Data Loading ====== #
args.plant = 126
args.spot = 174
args.start_date = "20190820"
args.end_date = "20200809"
args.batch_size = 64
args.x_frames = 3
args.y_frames = 1

# ====== Model Capacity ===== #
args.hid_dim = 256

# ====== Optimizer & Training ====== #
args.optim = 'RMSprop'
args.activation = 'relu'
args.lss = 'MSE'
args.lr = 0.001
args.epochs = 256
args.early_stop = 30
args.evaluation = 'NRMSE'

# ====== Experiment Variable ====== #
name_var1 = 'features'
list_var1 = [FeatureType.SUNSHINE,
                 FeatureType.HUMIDITY,
                 FeatureType.WIND_SPEED,
                 FeatureType.VISIBILITY,
                 FeatureType.GROUND_TEMPERATURE,
                 FeatureType.WIND_DIRECTION,
                 FeatureType.STEAM_PRESSURE,
                 FeatureType.TEMPERATURE,
                 FeatureType.PRECIPITATION,
                 FeatureType.DEW_POINT_TEMPERATURE,
                 FeatureType.ATMOSPHERIC_PRESSURE]

for i in range(len(list_var1)):
    sub_list = list_var1[:i + 1]

    setattr(args, name_var1, str(sub_list))
    print(args)

    setattr(args, name_var1, sub_list)   
    setting, model, result = experiment(deepcopy(args))
    save_exp_model(setting, model)
    save_exp_result(setting, result)
    print(result)
    print()

Namespace(activation='relu', batch_size=64, early_stop=30, end_date='20200809', epochs=256, evaluation='NRMSE', exp_name='solar power prediction using weather features', features="[<FeatureType.SUNSHINE: '일조(hr)'>]", hid_dim=256, lr=0.001, lss='MSE', optim='RMSprop', plant=126, spot=174, start_date='20190820', x_frames=3, y_frames=1)


/home/vislab/.conda/envs/tf-gpu/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Epoch 1/256
5/5 [==============================] - 1s 110ms/step - loss: 1317137152.0000 - accuracy: 0.5381 - val_loss: 855704256.0000 - val_accuracy: 0.3859
Epoch 2/256
5/5 [==============================] - 0s 11ms/step - loss: 1317137152.0000 - accuracy: 0.5381 - val_loss: 855704256.0000 - val_accuracy: 0.3859
Epoch 3/256
5/5 [==============================] - 0s 11ms/step - loss: 1317137152.0000 - accuracy: 0.5381 - val_loss: 855704256.0000 - val_accuracy: 0.3859
Epoch 4/256
5/5 [==============================] - 0s 11ms/step - loss: 1317137152.0000 - accuracy: 0.5381 - val_loss: 855704256.0000 - val_accuracy: 0.3859
Epoch 5/256
5/5 [==============================] - 0s 11ms/step - loss: 1317137024.0000 - accuracy: 0.5381 - val_loss: 855704256.0000 - val_accuracy: 0.3859
Epoch 6/256
5/5 [==============================] - 0s 11ms/step - loss: 1317137152.0000 - accuracy: 0.5381 - val_loss: 855704256.0000 - val_accuracy: 0.3859
Epoch 7/256
5/5 [==============================] - 0s 11m

<ipython-input-4-2dfd02194b65>:7: UserWarning: DEPRECATED: skimage.measure.compare_nrmse has been moved to skimage.metrics.normalized_root_mse. It will be removed from skimage.measure in version 0.18.
  nrmse = compare_nrmse(y_test, y_pred)


{'train_losses': [1317137152.0, 1317137152.0, 1317137152.0, 1317137152.0, 1317137024.0, 1317137152.0, 1317137152.0, 1317137152.0, 1317137024.0, 1317137024.0, 1317137152.0, 1317137024.0, 1317137152.0, 1317137152.0, 1317137152.0, 1317137152.0, 1317137152.0, 1317137152.0, 1317137152.0, 1317137152.0, 1317137280.0, 1317137152.0, 1317137152.0, 1317137152.0, 1317137152.0, 1317137152.0, 1317137024.0, 1317137152.0, 1317137152.0, 1317137024.0, 1317137152.0, 1317137280.0, 1317137024.0, 1317137024.0, 1317137152.0], 'val_losses': [855704256.0, 855704256.0, 855704256.0, 855704256.0, 855704256.0, 855704256.0, 855704256.0, 855704256.0, 855704256.0, 855704256.0, 855704256.0, 855704256.0, 855704256.0, 855704256.0, 855704256.0, 855704256.0, 855704256.0, 855704256.0, 855704256.0, 855704256.0, 855704256.0, 855704256.0, 855704256.0, 855704256.0, 855704256.0, 855704256.0, 855704256.0, 855704256.0, 855704256.0, 855704256.0, 855704256.0, 855704256.0, 855704256.0, 855704256.0, 855704256.0], 'train_accs': [0.538

...

Epoch 1/256
5/5 [==============================] - 1s 148ms/step - loss: 1316860416.0000 - accuracy: 0.1341 - val_loss: 854982976.0000 - val_accuracy: 0.0000e+00
Epoch 2/256
5/5 [==============================] - 0s 11ms/step - loss: 1316373632.0000 - accuracy: 0.0000e+00 - val_loss: 854868928.0000 - val_accuracy: 0.0000e+00
Epoch 3/256
5/5 [==============================] - 0s 11ms/step - loss: 1316287104.0000 - accuracy: 0.0000e+00 - val_loss: 854805760.0000 - val_accuracy: 0.0000e+00
Epoch 4/256
5/5 [==============================] - 0s 11ms/step - loss: 1316227328.0000 - accuracy: 0.0000e+00 - val_loss: 854755840.0000 - val_accuracy: 0.0000e+00
Epoch 5/256
5/5 [==============================] - 0s 11ms/step - loss: 1316179072.0000 - accuracy: 0.0000e+00 - val_loss: 854707648.0000 - val_accuracy: 0.0000e+00
Epoch 6/256
5/5 [==============================] - 0s 11ms/step - loss: 1316132096.0000 - accuracy: 0.0000e+00 - val_loss: 854662464.0000 - val_accuracy: 0.0000e+00
Epoch 7/256
5

5/5 [==============================] - 0s 11ms/step - loss: 1314227968.0000 - accuracy: 0.0000e+00 - val_loss: 852747776.0000 - val_accuracy: 0.0000e+00
Epoch 51/256
5/5 [==============================] - 0s 11ms/step - loss: 1314183808.0000 - accuracy: 0.0000e+00 - val_loss: 852705856.0000 - val_accuracy: 0.0000e+00
Epoch 52/256
5/5 [==============================] - 0s 11ms/step - loss: 1314142208.0000 - accuracy: 0.0000e+00 - val_loss: 852662464.0000 - val_accuracy: 0.0000e+00
Epoch 53/256
5/5 [==============================] - 0s 11ms/step - loss: 1314098432.0000 - accuracy: 0.0000e+00 - val_loss: 852615168.0000 - val_accuracy: 0.0000e+00
Epoch 54/256
5/5 [==============================] - 0s 11ms/step - loss: 1314052096.0000 - accuracy: 0.0000e+00 - val_loss: 852573248.0000 - val_accuracy: 0.0000e+00
Epoch 55/256
5/5 [==============================] - 0s 11ms/step - loss: 1314010112.0000 - accuracy: 0.0000e+00 - val_loss: 852529664.0000 - val_accuracy: 0.0000e+00
Epoch 56/256
5/5 

Epoch 100/256
5/5 [==============================] - 0s 11ms/step - loss: 1312056192.0000 - accuracy: 0.0066 - val_loss: 850565120.0000 - val_accuracy: 0.0060
Epoch 101/256
5/5 [==============================] - 0s 11ms/step - loss: 1312013824.0000 - accuracy: 0.0077 - val_loss: 850519296.0000 - val_accuracy: 0.0159
Epoch 102/256
5/5 [==============================] - 0s 11ms/step - loss: 1311968768.0000 - accuracy: 0.0102 - val_loss: 850477376.0000 - val_accuracy: 0.0228
Epoch 103/256
5/5 [==============================] - 0s 11ms/step - loss: 1311926784.0000 - accuracy: 0.0049 - val_loss: 850433728.0000 - val_accuracy: 0.0417
Epoch 104/256
5/5 [==============================] - 0s 11ms/step - loss: 1311883520.0000 - accuracy: 0.0138 - val_loss: 850390848.0000 - val_accuracy: 0.0417
Epoch 105/256
5/5 [==============================] - 0s 12ms/step - loss: 1311840768.0000 - accuracy: 0.0137 - val_loss: 850351808.0000 - val_accuracy: 0.0615
Epoch 106/256
5/5 [===========================

5/5 [==============================] - 0s 11ms/step - loss: 1317137152.0000 - accuracy: 0.5381 - val_loss: 855704256.0000 - val_accuracy: 0.3859
Epoch 152/256
5/5 [==============================] - 0s 11ms/step - loss: 1317137152.0000 - accuracy: 0.5381 - val_loss: 855704256.0000 - val_accuracy: 0.3859
Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 72, 10)]     0                                            
__________________________________________________________________________________________________
lstm_18 (LSTM)                  [(None, 256), (None, 273408      input_10[0][0]                   
__________________________________________________________________________________________________
repeat_vector_9 (RepeatVector)  (None, 24, 256)      0           lstm_18[0][0]       

Epoch 1/256
5/5 [==============================] - 1s 168ms/step - loss: 1317137152.0000 - accuracy: 0.5381 - val_loss: 855704256.0000 - val_accuracy: 0.3859
Epoch 2/256
5/5 [==============================] - 0s 11ms/step - loss: 1317137152.0000 - accuracy: 0.5381 - val_loss: 855704256.0000 - val_accuracy: 0.3859
Epoch 3/256
5/5 [==============================] - 0s 11ms/step - loss: 1317137152.0000 - accuracy: 0.5381 - val_loss: 855704256.0000 - val_accuracy: 0.3859
Epoch 4/256
5/5 [==============================] - 0s 11ms/step - loss: 1317137152.0000 - accuracy: 0.5381 - val_loss: 855704256.0000 - val_accuracy: 0.3859
Epoch 5/256
5/5 [==============================] - 0s 11ms/step - loss: 1317137024.0000 - accuracy: 0.5381 - val_loss: 855704256.0000 - val_accuracy: 0.3859
Epoch 6/256
5/5 [==============================] - 0s 11ms/step - loss: 1317137152.0000 - accuracy: 0.5381 - val_loss: 855704256.0000 - val_accuracy: 0.3859
Epoch 7/256
5/5 [==============================] - 0s 11m